In [ ]:
import sys; sys.path.append('..')
import numpy as np
import field_sampler, mesh

In [ ]:
V = [[0, 0], [1, 0], [0, 1], [1, 1], [1.5, 0], [0.5, -0.5]]
F = [[0, 1, 2], [1, 3, 2], [1, 4, 3], [1, 0, 5]]
m = mesh.Mesh(V, F, degree=2, embeddingDimension=2)

In [ ]:
# Define a scalar field on the vertices/nodes/elements
def testScalarField(X): return np.cos(X[:, 0] * 1 * np.pi) * np.cos(X[:, 1] * 1 * np.pi)
vertex_values = testScalarField(m.vertices())
node_values   = testScalarField(m.nodes())
element_values= testScalarField(m.vertices()[m.elements()].mean(axis=1)) # sample at barycenters

In [ ]:
sraw  = field_sampler.FieldSampler(V, F)
smesh = field_sampler.FieldSampler(m)

In [ ]:
# Construct a high-resolution sampling grid
X, Y = np.mgrid[-0.5:2.0:1001j, -1.0:1.5:1001j]
P = np.column_stack((X.ravel(), Y.ravel()))

In [ ]:
Craw  =  sraw.sample(P,  vertex_values)
Crawc =  sraw.sample(P, element_values)
Cquad = smesh.sample(P,    node_values)
Ccnst = smesh.sample(P, element_values)
# Restrict plot to the mesh domain
Craw [ ~sraw.contains(P)] = np.nan
Crawc[ ~sraw.contains(P)] = np.nan
Cquad[~smesh.contains(P)] = np.nan
Ccnst[~smesh.contains(P)] = np.nan

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(8, 8))
plt.subplot(2, 2, 1); plt.pcolormesh(X, Y,  Craw.reshape((X.shape[0], -1)), vmin=-1, vmax=1); plt.title('Raw Mesh (Linear)')
plt.subplot(2, 2, 2); plt.pcolormesh(X, Y, Crawc.reshape((X.shape[0], -1)), vmin=-1, vmax=1); plt.title('Raw Mesh (Constant)')
plt.subplot(2, 2, 3); plt.pcolormesh(X, Y, Cquad.reshape((X.shape[0], -1)), vmin=-1, vmax=1); plt.title('FEMMesh (Quadratic)')
plt.subplot(2, 2, 4); plt.pcolormesh(X, Y, Ccnst.reshape((X.shape[0], -1)), vmin=-1, vmax=1); plt.title('FEMMesh (Constant)')
plt.tight_layout()